In [124]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime
import time

In [125]:
df_payments = pd.read_csv(r"C:\Users\Lad\Downloads\Negativos - Pagamentos_download.csv")
df_installments = pd.read_csv(r"C:\Users\Lad\Downloads\Negativos - Parcelas_download.csv")
df_installments = df_installments.drop(columns="index")
df_payments = df_payments.drop(columns="key")

#df_installments["valor_a_conciliar"] = df_installments["valor_a_conciliar"].str.replace(",", ".").astype(float)
#df_installments["valor_parcela"] = df_installments["valor_parcela"].str.replace(",", ".").astype(float)
#df_payments["valor_bolepix"] = df_payments["valor_bolepix"].str.replace(",", ".").astype(float)
#df_installments["data_parcela"] = df_installments["data_parcela"].apply(lambda d: datetime.strptime(d, "%d/%m/%Y"))
#df_payments["settlement_date"] = df_payments["settlement_date"].apply(lambda d: datetime.strptime(d, "%d/%m/%Y"))

In [126]:
# payments list [(date, payment_value)]
# installments list [(installment_id, remaining_value)]

In [127]:
def conciliate_bolepix(client_name):
    
    payments_list = [(row["settlement_date"], row["valor_bolepix"]) 
                 for _, row in df_payments[df_payments["name"] == client_name].sort_values('settlement_date').iterrows()]
    installments_list = [(row["installment_id"], row["valor_a_conciliar"]) 
                 for _, row in df_installments[df_installments["nome"] == client_name].sort_values('data_parcela').iterrows()]
    
    conciliation_list = []
    #print(installments_list)
    while len(payments_list) > 0 and len(installments_list):
        current_payment_date, current_payment_value = payments_list[0]
        current_installment_id, current_installment_value = installments_list[0]


        if current_payment_value > current_installment_value:
            remaining_payment_value = current_payment_value - current_installment_value

            conciliation_list.append({
                "installment_id": current_installment_id,
                "client_name": client_name,
                "payment_date": current_payment_date,
                "payment_value": current_installment_value
            })
            installments_list.pop(0)
            payments_list[0] = (current_payment_date, remaining_payment_value)

        elif current_payment_value < current_installment_value:
            remaining_installment_value = current_installment_value - current_payment_value

            conciliation_list.append({
                "installment_id": current_installment_id,
                "client_name": client_name,
                "payment_date": current_payment_date,
                "payment_value": current_payment_value
            })
            payments_list.pop(0)
            installments_list[0] = (current_installment_id, remaining_installment_value)

        else:
            conciliation_list.append({
                "installment_id": current_installment_id,
                "client_name": client_name,
                "payment_date": current_payment_date,
                "payment_value": current_payment_value
            })
            payments_list.pop(0)
            installments_list.pop(0)

        #print(len(payments_list), len(installments_list), conciliation_list)

    return pd.DataFrame(conciliation_list)

In [128]:
df_conciliation = pd.concat(
    [conciliate_bolepix(name) for name in df_payments["name"].unique()]
)

In [129]:
df_conciliation.reset_index(drop=True).head(20)

,installment_id,client_name,payment_date,payment_value
0,94f8fcac-ef65-4856-8b52-b8c6f8277f78,ATELIER - COMERCIO DE TINTAS LTDA,2022-04-04,0
1,5ad65772-2600-443c-b016-4ab959bc926d,ATELIER - COMERCIO DE TINTAS LTDA,2022-04-04,0
2,104ba28b-2430-43a6-ae12-8c7235475a72,ATELIER - COMERCIO DE TINTAS LTDA,2022-04-04,17342
3,6308d0fc-9181-478b-828e-fab0b2b69a01,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,58773
4,2bbef0a0-dbc7-4064-837d-41684b8c3a7e,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,48040
5,ae74e25e-53e5-4183-8b83-e4f2e146d4b1,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,58773
6,c02cd945-6698-4aba-9083-011523ec1ba8,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,12878
7,8fa4534f-ec0e-407b-b856-bdabe11684e4,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,48040
8,9d03e7f3-676c-4f49-83cb-14014b3b1d2d,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,28795
9,2561c713-0863-4371-97c5-d3a840dbec77,ELZA MARIA DE OLIVEIRA PEREIRA 87510782449,2022-02-01,55591


In [130]:
ts = time.time()
ts = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d_%H_%M_%S')
df_conciliation.to_csv(r'C:\Users\Lad\Downloads\bolepix_conciliations'+str(ts)+'.csv')